In [19]:
import pandas as pd
import numpy as np
import re
import sys # Para leer las líneas malas
from datetime import date
from datetime import datetime
import csv

# Revisar primero el .txt con SublimeText

1) Si el separador es la barra vertical buscar ("| ) o (|") o (| ) o ( |) y reemplazarolo por ("|").

2) Buscar todas las tabulaciones ("\t") y reemplazarlas por espacios simples.

# Definir path del archivo y campos

In [20]:
path_matriculas_refeps = 'df_sucios/Bajada Matriculas 20220719.txt'
fecha_query = path_matriculas_refeps[-13:-4] #Calculo la fecha en base al path

In [21]:
#Conformo la fecha para después calcular las edades
fecha= pd.to_datetime(fecha_query,
                      yearfirst=False,
                      dayfirst=True,
                      infer_datetime_format=True)

In [22]:
fecha

Timestamp('2022-07-19 00:00:00')

In [23]:
#dictOfWords = { i : cols_wrang[i] for i in range(0, len(cols_wrang) ) }

In [24]:
cols_wrang = {1:'ID_PROFESIONAL',
2:'CODIGO',
3:'NOMBRE',
4:'APELLIDO',
5:'ID_TIPODOC',
6:'NRODOC',
7:'FECHA_NACIMIENTO',
8:'ID_LOC_NACIMIENTO',
9:'LOC_NACIMIENTO',
10:'ID_PAIS_NACIMIENTO',
11:'PAIS_NACIMIENTO',
12:'SEXO',
13:'FALLECIDO',
14:'FECHA_FALLECIDO',
15:'CUIL',
16:'ACTIVO',
17:'ID_CMDB_PERSONA',
18:'ID_PROVINCIA',
19:'PROVINCIA_CMDB',
20:'ID_LOCALIDAD',
21:'LOCALIDAD',
22:'DEPARTAMENTO',
23:'ID_PROFESIONAL_PROFESION',
24:'ID_PROFESION_REFERENCIA',
25:'PROFESION_REFERENCIA',
26:'ID_INSTITUCION_FORMADORA',
27:'FECHA_TITULO',
28:'ID_PROFESIONAL_MATRICULA',
29:'MATRICULA',
30:'FECHA_MATRICULA',
31:'ID_PROVINCIA_1',
32:'PROVINCIA_MATRICULA',
33:'ID_SITUACION_MATRICULA',
34:'ID_PROFESIONAL_SANCION',
35:'COLEGIO_REGISTRA',
36:'REMATRICULACION',
37:'COLEGIO_EMITE',
38:'ID_PROFESION',
39:'PROFESION',
40:'ID_LOCALIDAD_1',
41:'LOCALIDAD_MAT',
42:'ID_ORIGEN_EMITE',
43:'ORIGEN_EMITE'}



Cuento las lineas erroneas sobre la base de los campos recibidos

In [25]:
with open(path_matriculas_refeps, "r", encoding='latin') as archivo:
    with open("errores.csv", "w") as salida:
        contador_bad_lines = 0
        for linea in archivo:
            linea_l = linea.split("|")
            if len(linea_l)!= len(cols_wrang):
                contador_bad_lines += 1
                salida.write(linea)
        print (contador_bad_lines)

0


In [26]:
file = open(path_matriculas_refeps, encoding='latin')

In [27]:
type(file)

_io.TextIOWrapper

In [28]:
csvreader = csv.reader(file, delimiter = "|")

header = []
header = next(csvreader)

In [29]:
header[21]

'ID_PROFESIONAL_PROFESION'

### Levanto el DF

Ojo: ID_PROVINCIA refiere a provincia RENAPER y ID_PROVINCIA_1 a provincia de matriculación

In [30]:
df_matriculas_wrang = pd.read_csv(path_matriculas_refeps, sep ='|',
                            encoding = 'latin', 
                            error_bad_lines= False,
                            low_memory=False,
                            dtype = {'ID_PROFESIONAL': np.int32, 'ID_PROFESIONAL_PROFESION': np.int32,
                                      'ID_PROFESIONAL_MATRICULA': np.int32})



#para ubicar si hay alguna fila con valores null en ID_PROFESIONAL

#df_matriculas_wrang[df_matriculas_wrang['ID_PROFESIONAL'].isnull()]

#regex para encontrar saltos de línea en editor de texto (Sublime Text) r= \v(?=\|)
                             

In [31]:
df_matriculas_wrang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1538370 entries, 0 to 1538369
Data columns (total 43 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID_PROFESIONAL            1538370 non-null  int32  
 1   CODIGO                    1538370 non-null  int64  
 2   NOMBRE                    1538370 non-null  object 
 3   APELLIDO                  1538369 non-null  object 
 4   ID_TIPODOC                1538370 non-null  int64  
 5   NRODOC                    1538370 non-null  int64  
 6   FECHA_NACIMIENTO          1535943 non-null  object 
 7   ID_LOC_NACIMIENTO         1013674 non-null  float64
 8   LOC_NACIMIENTO            1012555 non-null  object 
 9   ID_PAIS_NACIMIENTO        1509049 non-null  float64
 10  SEXO                      1538370 non-null  object 
 11  FALLECIDO                 1538370 non-null  object 
 12  FECHA_FALLECIDO           92368 non-null    object 
 13  CUIL                      1

Renombro las columnas de PROVINCIA_RENAPER y PROVINCIA_MATRICULA

In [32]:
df_matriculas_wrang.rename({'ID_PROVINCIA':'ID_PROVINCIA_RENAPER', 'ID_PROVINCIA_1':'ID_PROVINCIA_MATRICULA'}, axis=1, inplace = True)

In [33]:
contador_post_parser = sum((df_matriculas_wrang.index.value_counts()))

In [34]:
print("Se deshecharon un",round(((contador_bad_lines/contador_post_parser)*100),4), "% de filas")

Se deshecharon un 0.0 % de filas


In [35]:
df_matriculas_wrang['ID_PROVINCIA_RENAPER'].unique()

array([ 2., nan,  1., 20.,  6., 23.,  5., 15.,  3.,  0., 14., 16.,  8.,
       13., 19., 21., 24., 10., 22., 12.,  7., 18.,  4., 17., 11.,  9.])

# Imputación de valores y eliminación de los NA

Reviso las PROVINCIAS_RENAPER que tengan datos no numéricos y les imputo NA

In [36]:
df_matriculas_wrang['ID_PROVINCIA_RENAPER'] = df_matriculas_wrang['ID_PROVINCIA_RENAPER'].apply(
                                        lambda x: np.nan if len(str(x)) > 4 else x)

Lleno las provincias RENAPER con 0 o '0', y les imputo NA

In [37]:
df_matriculas_wrang['ID_PROVINCIA_RENAPER'].loc[(df_matriculas_wrang['ID_PROVINCIA_RENAPER'] == '0') | 
                                                (df_matriculas_wrang['ID_PROVINCIA_RENAPER'] == 0)] = np.nan

/home/oblitterator/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [38]:
df_matriculas_wrang['ID_PROVINCIA_RENAPER'].unique()

array([ 2., nan,  1., 20.,  6., 23.,  5., 15.,  3., 14., 16.,  8., 13.,
       19., 21., 24., 10., 22., 12.,  7., 18.,  4., 17., 11.,  9.])

Ahora lleno esas NA con la provincia de matriculación, ordenada por última fecha de matriculación

In [39]:
df_matriculas_wrang['ID_PROVINCIA_RENAPER'] = df_matriculas_wrang.sort_values(['FECHA_MATRICULA'])['ID_PROVINCIA_RENAPER'].fillna(
    df_matriculas_wrang['ID_PROVINCIA_MATRICULA'])


In [40]:
df_matriculas_wrang['ID_PROVINCIA_RENAPER'].unique()

array([ 2.,  1., 20.,  6., 23.,  5., 15.,  3., 16., 14.,  8., 13., 19.,
       21., 24., 18., 10., 22., 12.,  7.,  4., 17., 11.,  9.])

In [41]:
df_matriculas_wrang[['ID_PROVINCIA_RENAPER',
                     'ID_PROVINCIA_MATRICULA']] = df_matriculas_wrang[['ID_PROVINCIA_RENAPER',
                                                                       'ID_PROVINCIA_MATRICULA']].astype(int)

Imputo NA para casos donde ID_CMDB_PERSONA sea no numérico

In [42]:
df_matriculas_wrang['ID_CMDB_PERSONA'] = df_matriculas_wrang['ID_CMDB_PERSONA'].apply(
                                        lambda x: np.nan if ((x != float) | (x != int)) else x)

# Defino columnas para nuevo Data Set

In [43]:
df_matriculas_wrang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1538370 entries, 0 to 1538369
Data columns (total 43 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID_PROFESIONAL            1538370 non-null  int32  
 1   CODIGO                    1538370 non-null  int64  
 2   NOMBRE                    1538370 non-null  object 
 3   APELLIDO                  1538369 non-null  object 
 4   ID_TIPODOC                1538370 non-null  int64  
 5   NRODOC                    1538370 non-null  int64  
 6   FECHA_NACIMIENTO          1535943 non-null  object 
 7   ID_LOC_NACIMIENTO         1013674 non-null  float64
 8   LOC_NACIMIENTO            1012555 non-null  object 
 9   ID_PAIS_NACIMIENTO        1509049 non-null  float64
 10  SEXO                      1538370 non-null  object 
 11  FALLECIDO                 1538370 non-null  object 
 12  FECHA_FALLECIDO           92368 non-null    object 
 13  CUIL                      1

In [44]:
df_matriculas_wrang.columns

Index(['ID_PROFESIONAL', 'CODIGO', 'NOMBRE', 'APELLIDO', 'ID_TIPODOC',
       'NRODOC', 'FECHA_NACIMIENTO', 'ID_LOC_NACIMIENTO', 'LOC_NACIMIENTO',
       'ID_PAIS_NACIMIENTO', 'SEXO', 'FALLECIDO', 'FECHA_FALLECIDO', 'CUIL',
       'ACTIVO', 'ID_CMDB_PERSONA', 'ID_PROVINCIA_RENAPER', 'PROVINCIA_CMDB',
       'ID_LOCALIDAD', 'LOCALIDAD', 'DEPARTAMENTO', 'ID_PROFESIONAL_PROFESION',
       'ID_PROFESION_REFERENCIA', 'PROFESION_REFERENCIA',
       'ID_INSTITUCION_FORMADORA', 'DENOMINACION', 'FECHA_TITULO',
       'ID_PROFESIONAL_MATRICULA', 'MATRICULA', 'FECHA_MATRICULA',
       'ID_PROVINCIA_MATRICULA', 'PROVINCIA_MATRICULA',
       'ID_SITUACION_MATRICULA', 'ID_PROFESIONAL_SANCION', 'COLEGIO_REGISTRA',
       'REMATRICULACION', 'COLEGIO_EMITE', 'ID_PROFESION', 'PROFESION',
       'ID_LOCALIDAD_1', 'LOCALIDAD_MAT', 'ID_ORIGEN_EMITE', 'ORIGEN_EMITE'],
      dtype='object')

In [45]:
cols_matriculas = ['ID_PROFESIONAL', 'CODIGO', 'NOMBRE', 'APELLIDO', 'ID_TIPODOC', #Tabla del Profesional
                   'NRODOC', 'FECHA_NACIMIENTO', 'ID_PAIS_NACIMIENTO', 'SEXO', 'CUIL', #Tabla del Profesional
                   'ID_PROVINCIA_RENAPER','PROVINCIA_CMDB', 'ID_CMDB_PERSONA',  #Tabla del ciudadano #Antes PROVINCIA_CMDB = NOMBRE_2
                   'FALLECIDO','FECHA_FALLECIDO', #Tabla del ciudadano
                   'ID_PROFESIONAL_PROFESION','ID_PROFESION_REFERENCIA', 'PROFESION_REFERENCIA', 'ID_INSTITUCION_FORMADORA','DENOMINACION', 'FECHA_TITULO', #Tabla de la profesión, antes PROFESION_REFERENCIA = NOMBRE_4
                   'ID_PROFESIONAL_MATRICULA', 'MATRICULA', 'FECHA_MATRICULA', 'REMATRICULACION', #Tabla de la matrícula
                   'ID_PROVINCIA_MATRICULA', 'PROVINCIA_MATRICULA', #Tabla de la matrícula, antes PROVINCIA_MATRICULA = NOMBRE_5
                   'ID_SITUACION_MATRICULA',  'COLEGIO_REGISTRA', 'ACTIVO', #'SSS',  #Tabla de la matrícula
                   'COLEGIO_EMITE', 'ID_ORIGEN_EMITE','ORIGEN_EMITE', #Tabla de la matrícula, antes ORIGEN_EMITE=NOMBRE_8
                   'ID_PROFESIONAL_SANCION'] #Tabla de las bajas y sanciones de la matrícula

In [46]:
df_matriculas_wrang = df_matriculas_wrang[cols_matriculas]

In [47]:
df_matriculas_wrang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1538370 entries, 0 to 1538369
Data columns (total 34 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID_PROFESIONAL            1538370 non-null  int32  
 1   CODIGO                    1538370 non-null  int64  
 2   NOMBRE                    1538370 non-null  object 
 3   APELLIDO                  1538369 non-null  object 
 4   ID_TIPODOC                1538370 non-null  int64  
 5   NRODOC                    1538370 non-null  int64  
 6   FECHA_NACIMIENTO          1535943 non-null  object 
 7   ID_PAIS_NACIMIENTO        1509049 non-null  float64
 8   SEXO                      1538370 non-null  object 
 9   CUIL                      1445373 non-null  float64
 10  ID_PROVINCIA_RENAPER      1538370 non-null  int64  
 11  PROVINCIA_CMDB            1164839 non-null  object 
 12  ID_CMDB_PERSONA           0 non-null        float64
 13  FALLECIDO                 1

In [48]:
df_matriculas_wrang[['ID_PAIS_NACIMIENTO',
                     'CUIL',
                     'ID_CMDB_PERSONA',
                     'ID_ORIGEN_EMITE',
                     'ID_SITUACION_MATRICULA',
                     'ID_PROFESIONAL_SANCION']] = df_matriculas_wrang[['ID_PAIS_NACIMIENTO',
                                                                       'CUIL',
                                                                       'ID_CMDB_PERSONA',
                                                                       'ID_ORIGEN_EMITE',
                                                                       'ID_SITUACION_MATRICULA',
                                                                       'ID_PROFESIONAL_SANCION']].fillna(0).astype(np.int64)

# Trabajo sobre las fechas

In [49]:
#Ver si se puede hacer en una sola línea de código

df_matriculas_wrang['FECHA_NACIMIENTO'] = pd.to_datetime(df_matriculas_wrang['FECHA_NACIMIENTO'],
                                                         format='%d/%m/%Y',
                                                         errors='coerce')

df_matriculas_wrang['FECHA_TITULO'] = pd.to_datetime(df_matriculas_wrang['FECHA_TITULO'],
                                                     format='%d/%m/%Y',
                                                     errors='coerce')

df_matriculas_wrang['FECHA_MATRICULA'] = pd.to_datetime(df_matriculas_wrang['FECHA_MATRICULA'],
                                                        format='%d/%m/%Y',
                                                        errors='coerce')

df_matriculas_wrang['FECHA_FALLECIDO'] = pd.to_datetime(df_matriculas_wrang['FECHA_FALLECIDO'],
                                                        format='%d/%m/%Y',
                                                        errors='coerce')

## Agrego el campo edad a la fecha de nacimiento

In [50]:
def age(birth_date):
    today = fecha
    y = today.year - birth_date.year
    if today.month < birth_date.month or today.month == birth_date.month and today.day < birth_date.day:
        y -= 1
    return y

In [51]:
df_matriculas_wrang['EDAD'] = df_matriculas_wrang['FECHA_NACIMIENTO'].apply(age)

# Convierto en booleanos los campos SI y NO

In [52]:
df_matriculas_wrang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1538370 entries, 0 to 1538369
Data columns (total 35 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   ID_PROFESIONAL            1538370 non-null  int32         
 1   CODIGO                    1538370 non-null  int64         
 2   NOMBRE                    1538370 non-null  object        
 3   APELLIDO                  1538369 non-null  object        
 4   ID_TIPODOC                1538370 non-null  int64         
 5   NRODOC                    1538370 non-null  int64         
 6   FECHA_NACIMIENTO          1535933 non-null  datetime64[ns]
 7   ID_PAIS_NACIMIENTO        1538370 non-null  int64         
 8   SEXO                      1538370 non-null  object        
 9   CUIL                      1538370 non-null  int64         
 10  ID_PROVINCIA_RENAPER      1538370 non-null  int64         
 11  PROVINCIA_CMDB            1164839 non-null  object

In [53]:
df_matriculas_wrang['FALLECIDO'] = df_matriculas_wrang['FALLECIDO'].fillna('NO').map(
                   {'SI':True ,'NO':False})
df_matriculas_wrang['REMATRICULACION'] = df_matriculas_wrang['REMATRICULACION'].fillna('NO').map(
                   {'SI':True ,'NO':False})         
df_matriculas_wrang['COLEGIO_REGISTRA'] = df_matriculas_wrang['COLEGIO_REGISTRA'].fillna('NO').map(
                   {'SI':True ,'NO':False})
df_matriculas_wrang['ACTIVO'] = df_matriculas_wrang['ACTIVO'].fillna('NO').map(
                   {'SI':True ,'NO':False})
df_matriculas_wrang['COLEGIO_EMITE'] = df_matriculas_wrang['COLEGIO_EMITE'].fillna('NO').map(
                   {'SI':True ,'NO':False})

In [54]:
df_matriculas_wrang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1538370 entries, 0 to 1538369
Data columns (total 35 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   ID_PROFESIONAL            1538370 non-null  int32         
 1   CODIGO                    1538370 non-null  int64         
 2   NOMBRE                    1538370 non-null  object        
 3   APELLIDO                  1538369 non-null  object        
 4   ID_TIPODOC                1538370 non-null  int64         
 5   NRODOC                    1538370 non-null  int64         
 6   FECHA_NACIMIENTO          1535933 non-null  datetime64[ns]
 7   ID_PAIS_NACIMIENTO        1538370 non-null  int64         
 8   SEXO                      1538370 non-null  object        
 9   CUIL                      1538370 non-null  int64         
 10  ID_PROVINCIA_RENAPER      1538370 non-null  int64         
 11  PROVINCIA_CMDB            1164839 non-null  object

In [55]:
df_matriculas_wrang['SEXO'].unique()

array(['F', 'M'], dtype=object)

# Métrica final entre cuantas filas entraron y cuantas desheché

In [56]:
contador_lineas_final = sum(df_matriculas_wrang.index.value_counts())

In [57]:
print("Se deshecharon un",round((((contador_post_parser-contador_lineas_final)/contador_lineas_final)*100),4), "% de filas")

Se deshecharon un 0.0 % de filas


# Exporto el Data Set Limpio

In [58]:
#path = r'C:/Users/Juan/REFEPS/Data_Wrangling/'

Guardo el DF limpio, recuperando la fecha de la query

In [59]:
df_matriculas_wrang.to_csv(("df_limpios/"+fecha_query+"_df_matriculas_limpio.csv"), encoding='latin', index = False)

In [60]:
df_matriculas_wrang.sort_values('FECHA_MATRICULA',
                                ascending = False)[['FECHA_MATRICULA',
                                                    'ID_PROFESIONAL']].drop_duplicates('ID_PROFESIONAL')

,FECHA_MATRICULA,ID_PROFESIONAL
493487,2027-09-19,20207471
1169529,2022-07-18,36049650
1260087,2022-07-18,44018409
1260085,2022-07-15,44018407
1428094,2022-07-15,62012828
...,...,...
1426969,NaT,62011716
1426970,NaT,62011717
1426971,NaT,62011718
1426972,NaT,62011719
